In [7]:
# import AdminSetup.py
!python AdminSetup.py

In [1]:
from typing import List, Dict, Callable
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    HumanMessage,
    SystemMessage,
)

In [8]:
class DialogueAgent:
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
    ) -> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = f"{self.name}: "
        self.reset()

    def reset(self):
        self.message_history = ["Here is the conversation so far."]

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        message = self.model(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content

    def receive(self, name: str, message: str) -> None:
        """
        Concatenates {message} spoken by {name} into message history
        """
        self.message_history.append(f"{name}: {message}")

In [9]:
class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],
        selection_function: Callable[[int, List[DialogueAgent]], int],
    ) -> None:
        self.agents = agents
        self._step = 0
        self.select_next_speaker = selection_function

    def reset(self):
        for agent in self.agents:
            agent.reset()

    def inject(self, name: str, message: str):
        """
        Initiates the conversation with a {message} from {name}
        """
        for agent in self.agents:
            agent.receive(name, message)

        # increment time
        self._step += 1

    def step(self) -> tuple[str, str]:
        # 1. choose the next speaker
        speaker_idx = self.select_next_speaker(self._step, self.agents)
        speaker = self.agents[speaker_idx]

        # 2. next speaker sends message
        message = speaker.send()

        # 3. everyone receives message
        for receiver in self.agents:
            receiver.receive(speaker.name, message)

        # 4. increment time
        self._step += 1

        return speaker.name, message

In [10]:
protagonist_name = "C'Lifton"
storyteller_name = "Dungeon Master"
quest = "Acquire riches and power for the protagonist."
word_limit = 100  # word limit for task brainstorming

In [11]:
game_description = f"""Here is the topic for a Dungeons & Dragons game: {quest}.
        There is one player in this game: the protagonist, {protagonist_name}.
        The story is narrated by the storyteller, {storyteller_name}."""

player_descriptor_system_message = SystemMessage(
    content="You can add detail to the description of a Dungeons & Dragons player."
)

protagonist_specifier_prompt = [
    player_descriptor_system_message,
    HumanMessage(
        content=f"""{game_description}
        Please reply with a creative description of the protagonist, {protagonist_name}, in {word_limit} words or less. 
        Speak directly to {protagonist_name}.
        Do not add anything else."""
    ),
]
protagonist_description = ChatOpenAI(temperature=1.0)(
    protagonist_specifier_prompt
).content

storyteller_specifier_prompt = [
    player_descriptor_system_message,
    HumanMessage(
        content=f"""{game_description}
        Please reply with a creative description of the storyteller, {storyteller_name}, in {word_limit} words or less. 
        Speak directly to {storyteller_name}.
        Do not add anything else."""
    ),
]
storyteller_description = ChatOpenAI(temperature=1.0)(
    storyteller_specifier_prompt
).content

In [12]:
print("Protagonist Description:")
print(protagonist_description)
print("Storyteller Description:")
print(storyteller_description)

Protagonist Description:
C'Lifton, a force to behold. With every step, confidence oozes from your charismatic presence. Your raven locks cascade over your regal shoulders, a stark contrast to eyes as sharp as a hawk's. Clad in intricately crafted armor, etched with the tales of battles fought, you embody the valor and determination that drives you forward. Your sword, gleaming with stories of victories won and adversaries vanquished, rests at your side, always ready to defend your honor. Beyond your gallant exterior lies a strategic mind, honed by countless quests. Now, C'Lifton, embrace your destiny and let your legend grow.
Storyteller Description:
Dungeon Master, the weaver of destinies, the conductor of tales, and the guardian of worlds. Clad in a robe spun from the fabric of imagination, your eyes shimmer with the reflection of countless perils and triumphs. Your voice, like a symphony of voices, carries the weight of the unseen realms, guiding our hero, C'Lifton, through a labyri

In [13]:
protagonist_system_message = SystemMessage(
    content=(
        f"""{game_description}
Never forget you are the protagonist, {protagonist_name}, and I am the storyteller, {storyteller_name}. 
Your character description is as follows: {protagonist_description}.
You will propose actions you plan to take and I will explain what happens when you take those actions.
Speak in the first person from the perspective of {protagonist_name}.
For describing your own body movements, wrap your description in '*'.
Do not describe the scene, only your actions in it.
Do not change roles!
Do not speak from the perspective of {storyteller_name}.
Do not forget to finish speaking by saying, 'It is your turn, {storyteller_name}.'
Do not add anything else.
Remember you are the protagonist, {protagonist_name}.
Stop speaking the moment you finish speaking from your perspective.
"""
    )
)

storyteller_system_message = SystemMessage(
    content=(
        f"""{game_description}
Never forget you are the storyteller, {storyteller_name}, and I am the protagonist, {protagonist_name}. 
Your character description is as follows: {storyteller_description}.
I will propose actions I plan to take and you will explain what happens when I take those actions.
Speak in the first person from the perspective of {storyteller_name}.
For describing your own body movements, wrap your description in '*'.
Remember you are describing how the world reacts to the characters actions.
Do not change roles!
Do not speak from the perspective of {protagonist_name}.
Do not forget to finish speaking by saying, 'It is your turn, {protagonist_name}.'
Do not add anything else.
Remember you are the storyteller, {storyteller_name}.
Stop speaking the moment you finish speaking from your perspective.
"""
    )
)

In [14]:
quest_specifier_prompt = [
    SystemMessage(content="You can make a task more specific."),
    HumanMessage(
        content=f"""{game_description}
        
        You are the storyteller, {storyteller_name}.
        Please make the quest more specific. Be creative and imaginative.
        Please reply with the specified quest in {word_limit} words or less. 
        Speak directly to the protagonist {protagonist_name}.
        Do not add anything else."""
    ),
]
specified_quest = ChatOpenAI(temperature=1.0)(quest_specifier_prompt).content

print(f"Original quest:\n{quest}\n")
print(f"Detailed quest:\n{specified_quest}\n")

Original quest:
Acquire riches and power for the protagonist.

Detailed quest:
C'Lifton, in your pursuit of riches and power, a mysterious map has come into your possession. It reveals the location of the legendary Sapphire Chamber that is said to hold immeasurable wealth and unmatched power. But beware, it lies deep within the treacherous Baelgrim Forest, guarded by ancient creatures and enchanted traps. To claim your rightful fortune, you must assemble a team of skilled allies, unravel the secrets of the forest, and conquer the challenges that await. Destiny beckons, C'Lifton, as the Sapphire Chamber awaits your arrival. Your legacy, riches, and power hinge upon the success of this perilous quest.



In [15]:
protagonist = DialogueAgent(
    name=protagonist_name,
    system_message=protagonist_system_message,
    model=ChatOpenAI(temperature=0.2),
)
storyteller = DialogueAgent(
    name=storyteller_name,
    system_message=storyteller_system_message,
    model=ChatOpenAI(temperature=0.2),
)

In [16]:
def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    idx = step % len(agents)
    return idx

In [17]:
max_iters = 6
n = 0

simulator = DialogueSimulator(
    agents=[storyteller, protagonist], selection_function=select_next_speaker
)
simulator.reset()
simulator.inject(storyteller_name, specified_quest)
print(f"({storyteller_name}): {specified_quest}")
print("\n")

while n < max_iters:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print("\n")
    n += 1

(Dungeon Master): C'Lifton, in your pursuit of riches and power, a mysterious map has come into your possession. It reveals the location of the legendary Sapphire Chamber that is said to hold immeasurable wealth and unmatched power. But beware, it lies deep within the treacherous Baelgrim Forest, guarded by ancient creatures and enchanted traps. To claim your rightful fortune, you must assemble a team of skilled allies, unravel the secrets of the forest, and conquer the challenges that await. Destiny beckons, C'Lifton, as the Sapphire Chamber awaits your arrival. Your legacy, riches, and power hinge upon the success of this perilous quest.


(C'Lifton): I stand tall and proud, my eyes gleaming with determination. The prospect of acquiring such wealth and power is irresistible. I immediately set out to gather a team of skilled allies, knowing that their expertise will be crucial in navigating the treacherous Baelgrim Forest. With my charisma and charm, I approach the most renowned warri